<center><h1><b>Mount Google Drive</b></h1></center>

In [1]:
#@title Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<center><h1><b>FFMPEG</b></h1></center>

In [ ]:
#@title Install FFMPEG 

!git clone https://github.com/XniceCraft/ffmpeg-colab.git
!chmod +x ./ffmpeg-colab/install
!./ffmpeg-colab/install
!rm -fr /content/ffmpeg-colab
!ffmpeg -version

In [ ]:
#@title Convert MKV to MP4 Input from Form, and automatic Output to Movie Dir (Create Movie Dir First on MyDrive)

import os, sys, re

video_file_path = "/content/drive/MyDrive/video.mkv" #@param {type:"string"}
output_name_video = "Output Video Name" #@param {type:"string"}

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = "/content/drive/MyDrive/Movie"
os.environ['fileName'] = output_name_video
os.environ['fileExtension'] = file_extension_raw

! ffmpeg -i "$inputFile" -codec copy -crf 27 -preset veryslow "$outputPath"/"$fileName"-[INFOKAN21].mp4

In [ ]:
#@title Remove Subtitle MKV
import os, sys, re

video_file_path = "/content/drive/MyDrive/video.mkv" #@param {type:"string"}
output_name_video = "Top Gun Maverick 2022-BLURAY" #@param {type:"string"}
reso = "720" #@param ["360", "480", "720", "1080"]

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = "/content/drive/MyDrive/MKV"
os.environ['fileName'] = output_name_video
os.environ['reso'] = reso

! ffmpeg -i "$inputFile" -sn -c copy "$outputPath"/"$fileName"-"$reso"-[INFOKAN21].mkv

In [ ]:
#@title Merge Subtitle MKV (Create subbed Dir First on MyDrive)

import os

video_file_path = "/content/drive/MyDrive/video.mkv" #@param {type:"string"}
subs_file_path = "/content/drive/MyDrive/sub.srt" #@param {type:"string"}
output_name_video = "Top Gun Maverick 2022-BLURAY" #@param {type:"string"}
reso = "720" #@param ["360", "480", "720", "1080"]

os.environ['inputFile'] = video_file_path
os.environ['inputSubsFile'] = subs_file_path
os.environ['outputPath'] = "/content/drive/MyDrive/subbed/"
os.environ['fileName'] = output_name_video
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -i "$inputSubsFile" -c:v copy -c:a copy "$outputPath"/"$fileName"-"$reso"-[INFOKAN21]-SubIndonesia.mkv

In [ ]:
#@title Merge Subtitle MP4 (Create subbed Dir First on MyDrive)

import os

video_file_path = "/content/drive/MyDrive/video.mkv" #@param {type:"string"}
subs_file_path = "/content/drive/MyDrive/sub.srt" #@param {type:"string"}
output_name_video = "Top Gun Maverick 2022-BLURAY" #@param {type:"string"}

os.environ['inputFile'] = video_file_path
os.environ['inputSubsFile'] = subs_file_path
os.environ['outputPath'] = "/content/drive/MyDrive/subbed/"
os.environ['fileName'] = output_name_video


! ffmpeg -i "$inputFile" -i "$inputSubsFile" -c copy -c:s mov_text -metadata:s:s:0 title="Indonesia" "$outputPath"/"$fileName"-[INFOKAN21]-SubIndonesia.mp4

<center><h1><b>Upload File</b></h1></center>

In [ ]:
#@title Upload to Bayfiles.com | Only mp4
import os, sys, re

video_file_path = "/content/drive/MyDrive/video.mp4" #@param {type:"string"}

os.environ['inputFile'] = video_file_path

! curl -F "file=@$inputFile" https://api.bayfiles.com/upload

In [ ]:
#@title Upload to Anonfiles.com | Only mp4
import os, sys, re

video_file_path = "/content/drive/MyDrive/video.mp4" #@param {type:"string"}

os.environ['inputFile'] = video_file_path

! curl -F "file=@$inputFile" https://api.anonfiles.com/upload

In [ ]:
#@title Upload to hellabyte.cloud
import os, sys, re

video_file_path = "/content/drive/MyDrive/video.mp4" #@param {type:"string"}
API = "API from your account" #@param {type:"string"}

os.environ['inputFile'] = video_file_path
os.environ['api'] = API

! curl -F "file=@$inputFile" up.hellabyte.cloud/"$api"

<center><h1><b>Extract and Compress</b></h1></center>

<p><b>Before Extract or Compress file, Mount Google Drive First on First Cell

In [ ]:
#@title Extract Zip

import os

compress_file = "/content/drive/MyDrive/file.zip" #@param {type:"string"}
output_folder = "Output folder where file will be extracted" #@param {type:"string"}

os.environ['inputFile'] = compress_file
os.environ['outputPath'] = output_folder

! unzip "$inputFile" -d "$outputPath"

In [ ]:
#@title Extract RAR (Without Password)

import os

compress_file = "/content/drive/MyDrive/file.rar" #@param {type:"string"}
output_folder = "Output folder where file will be extracted" #@param {type:"string"}

os.environ['inputFile'] = compress_file
os.environ['outputPath'] = output_folder

!unrar x "$inputFile" "$outputPath"

In [3]:
#@title ZIP file/folder

import os

file_folder_name = "/content/drive/MyDrive/Movie/Ghost Writer 2 2022-WEBDL-[INFOKAN21].mp4" #@param {type:"string"}
output_file_name = "Ghst.Wrtr2022-WEBDL-[INFOKAN21]" #@param {type:"string"}

os.environ['inputFile'] = file_folder_name
os.environ['outputFile'] = output_file_name

!zip /content/drive/MyDrive/"$outputFile".zip "$inputFile"

  adding: content/drive/MyDrive/Movie/Ghost Writer 2 2022-WEBDL-[INFOKAN21].mp4 (deflated 0%)


<center><h1><b>Google Drive To Mega</b></h1></center>

In [ ]:
# @title Input your Mega Id and Password

# https://github.com/surajbahadur591

from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = "Email Login Mega"  # @param {type:"string"}
PASSWORD = "Password Mega"  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")

In [ ]:
import time
import subprocess
import contextlib
from IPython.display import clear_output
# @title <center><h2>TRANSFER NOW</h2></center><br>
# TRANSFER NOW

PATH_TO_FILE = "/content/drive/MyDrive/Video.mp4"  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    cmd = ['mega-put', PATH_TO_FILE]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")

<center><h1><b>ADDITIONAL CELL</b></h1></center>

# Colab Keep Alive Console Script

<p><font size=2px ><font color="red"> Tip - Set a javascript interval to click on the connect button every 60 seconds. Open developer-settings (in your web-browser) with Ctrl+Shift+I then click on console tab and type this on the console prompt. (for mac press Option+Command+I)</font></p><b>Copy script in hidden cell and paste at your browser console !!! DO NOT CLOSE YOUR BROWSER IN ORDER TO STILL RUNNING SCRIPT</b>


<code>function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-connect-button").click() 
}setInterval(ClickConnect,6000)</code>